In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data1 = np.load('predictive_train2.npz', allow_pickle=True)
train_input = data1['inputs'].astype(float)
train_targets = data1['targets'].astype(float)

data2 = np.load('predictive_validation2.npz', allow_pickle=True)
validation_input = data1['inputs'].astype(float)
validation_targets = data1['targets'].astype(float)

data3 = np.load('predictive_test2.npz', allow_pickle=True)
test_input = data1['inputs'].astype(float)
test_targets = data1['targets'].astype(float)

In [3]:
n_samples = len(train_targets)
classes = np.unique(train_targets)
n_classes = len(classes)

classweight = {}

for cls in classes:
    n_cls = np.sum(train_targets == cls)
    classweight[cls] = n_samples / (n_cls * n_classes)


In [4]:
output_layer_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_layer_size, activation='softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [7]:

validation_input = validation_input.reshape(-1)
test_input = test_input.reshape(-1)

In [8]:
epoch = 100
early_stop = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights=True)


model.fit(
    train_input,
    train_targets,
    batch_size = 100,
    class_weight = classweight,
    epochs = epoch,
    callbacks = early_stop,
    validation_data = (validation_input, validation_targets),
    verbose = 1 
)

Epoch 1/100


ValueError: Dimensions must be equal, but are 100 and 200 for '{{node LogicalAnd_1}} = LogicalAnd[](Tile_2, Greater)' with input shapes: [1,100], [1,200].

In [ ]:
print(train_input.shape)
print(train_targets.shape)

(64000,)
(8000, 1)


In [ ]:
test_input.shape

(64000,)